In [1]:
%matplotlib inline
import os
import glob
import csv
import re
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import pywt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from mpl_toolkits.mplot3d import Axes3D 
from scipy import signal
from scipy.signal import find_peaks
from scipy import optimize
from scipy.stats import linregress, stats
import seaborn as sns

import datetime as dt
from textwrap import wrap


# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
os.getcwd()

'C:\\Users\\bina_\\Dropbox\\Robina Shaheen_PEMS2022\\PEMS_NOx_BC_CO2_ALPR\\ii.dwt'

In [2]:
df1 = pd.read_csv('./Results2/CO2_s6_HWpeak.csv', )
df = pd.read_csv('./Results2/full_CO2_HWpeak.csv', )
del df1[df1.columns[0]]
# df2=df.transpose()
del df[df.columns[0]]
print(df1.head())
df.head()

            1           2           3           4            5            6  \
0   38.526629  113.538243   69.130348   11.339629    31.215219   369.306691   
1  367.884608  367.910378  376.305601  373.332662   377.482472   375.427739   
2  406.558750  541.237969  798.237072  928.733648  1015.272093   986.957097   
3  445.085379  654.776212  867.367420  940.073276  1046.487312  1356.263788   

             7            8            9           10  ...           476  \
0    72.552164    45.571935    80.787992     6.126556  ...     54.340542   
1   377.484424   375.835298   378.049224   382.935331  ...    362.428604   
2  1499.715378  1623.523741  1795.052460  2013.056829  ...  69336.365486   
3  1572.267542  1669.095676  1875.840452  2019.183384  ...  69390.706028   

            477           478           479           480           481  \
0     70.159251     75.218712     54.923722     93.687568    120.799439   
1    362.234363    366.294560    362.613754    369.189216    385.345512  

,0,1,2,3,4,5,6,7,8,9,...,476,477,478,479,480,481,482,483,484,485
0,126.247273,38.526629,113.538243,69.130348,11.339629,31.215219,369.306691,72.552164,45.571935,80.787992,...,54.340542,70.159251,75.218712,54.923722,93.687568,120.799439,86.531491,256.934368,49.400198,16.145109
1,368.902634,367.884608,367.910378,376.305601,373.332662,377.482472,375.427739,377.484424,375.835298,378.049224,...,362.428604,362.234363,366.294560,362.613754,369.189216,385.345512,368.002714,367.393722,370.828686,365.388356
2,230.964721,406.558750,541.237969,798.237072,928.733648,1015.272093,986.957097,1499.715378,1623.523741,1795.052460,...,69336.365486,69517.217579,69651.637849,69783.402376,69999.192751,70163.469667,70451.978287,70610.466956,70805.624716,71033.615481
3,357.211993,445.085379,654.776212,867.367420,940.073276,1046.487312,1356.263788,1572.267542,1669.095676,1875.840452,...,69390.706028,69587.376830,69726.856561,69838.326098,70092.880319,70284.269106,70538.509778,70867.401324,70855.024914,71049.760590


In [3]:
df2=df.transpose()

df2.head()

,0,1,2,3
0,126.247273,368.902634,230.964721,357.211993
1,38.526629,367.884608,406.558750,445.085379
2,113.538243,367.910378,541.237969,654.776212
3,69.130348,376.305601,798.237072,867.367420
4,11.339629,373.332662,928.733648,940.073276


In [4]:
df2.rename(columns={0:'HW_CO2', 1:'peak_height', 
                    2: 'Ti',3: 'Tf'  }, inplace=True)
df2.head(3)

,HW_CO2,peak_height,Ti,Tf
0,126.247273,368.902634,230.964721,357.211993
1,38.526629,367.884608,406.558750,445.085379
2,113.538243,367.910378,541.237969,654.776212


In [5]:
df2["Area_CO2"] = df2["HW_CO2"] * df2['peak_height']
df2.head()

,HW_CO2,peak_height,Ti,Tf,Area_CO2
0,126.247273,368.902634,230.964721,357.211993,46572.951467
1,38.526629,367.884608,406.558750,445.085379,14173.353748
2,113.538243,367.910378,541.237969,654.776212,41771.897940
3,69.130348,376.305601,798.237072,867.367420,26014.137202
4,11.339629,373.332662,928.733648,940.073276,4233.453754


In [6]:
df2["CO2 (ug/m3)"] = (df2["Area_CO2"] * 44)/24.45
print(len(df2))

df2.to_csv('./Results2/CO2_peakarea_index.csv')
df2.head()

486


,HW_CO2,peak_height,Ti,Tf,Area_CO2,CO2 (ug/m3)
0,126.247273,368.902634,230.964721,357.211993,46572.951467,83812.264399
1,38.526629,367.884608,406.558750,445.085379,14173.353748,25506.239873
2,113.538243,367.910378,541.237969,654.776212,41771.897940,75172.331671
3,69.130348,376.305601,798.237072,867.367420,26014.137202,46814.807235
4,11.339629,373.332662,928.733648,940.073276,4233.453754,7618.485284


In [ ]:
# # Functions --------------------------------------------------------------------------
# def process_df(df):
#     """
#     You could write your dataframe processing into this function.
#     """
#     out_df = df # as a placeholder, I'm just making the output_df = df. 
#     return out_df

# def chunker(df, size):
#     """
#     Split dataframe into chunks with specified number of rows
#     Args:
#         df: Input Pandas DataFrame or DataSeries
#         size: Number of rows per chunk
#     Returns:
#         Pandas DataFrame: the chunk
#     """
#     for pos in range(0, len(df), size):
#         yield df.iloc[pos:pos + size] 

In [ ]:
# # Process dataframe in chunks ------------------------------------------------
# chunk_size = 6      # specify number of rows per chunk (you will want to modify this to suit your needs)
# li = []             # placeholder that will hold the chunked results

# for i in chunker(fake_df, chunk_size):
#     result = process_df(i)
#     li.append(result)

# final_output = pd.concat(li)  # append all the chunked results together into a final dataframe